[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Johannes-Steinle/Small_Language_Models/blob/main/notebooks/SLM_Finetuning_Demo.ipynb)

# SLM Fine-Tuning & Inferenz Demo: Gemma 3 (QLoRA)

Dieses Notebook zeigt den kompletten Workflow eines Small Language Models (SLM):
1. **Modell laden** — Gemma-3-4B in 4-Bit-Quantisierung (~3–4 GB VRAM)
2. **VORHER-Test** — Antwort des Basismodells auf eine Fachfrage
3. **Fine-Tuning** — Anpassung mittels QLoRA auf einem deutschen Instruktions-Datensatz
4. **NACHHER-Test** — Dieselbe Frage erneut stellen und die Antwort vergleichen

**Voraussetzungen:**
1. Hugging Face Account
2. Akzeptierte Gemma-3-Lizenzbedingungen auf Hugging Face
3. HF Access Token (Read)

In [ ]:
# 1. Abhängigkeiten installieren
!pip install -q -U bitsandbytes transformers peft accelerate trl datasets

In [ ]:
# 2. Bei Hugging Face anmelden
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# 3. Modell laden (4-Bit-Quantisierung)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Gemma-3-4B in 4-Bit benötigt ~3–4 GB VRAM — passt auf Colab T4.
model_id = "google/gemma-3-4b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = "right"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

print(f"VRAM-Verbrauch: {torch.cuda.memory_allocated()/1024**3:.1f} GB")

# Hilfsfunktion für Inferenz (funktioniert vor und nach dem Training)
def generate(prompt, max_tokens=512):
    model.eval()
    text = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    input_len = inputs["input_ids"].shape[-1]
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_tokens, do_sample=False)
    return tokenizer.decode(out[0][input_len:], skip_special_tokens=True)

In [ ]:
# 4. VORHER-Test — Antwort des Basismodells
test_prompt = "Was ist der Unterschied zwischen LoRA und vollem Fine-Tuning?"
print(f"Prompt: {test_prompt}\n")
before_text = generate(test_prompt)
print("VORHER-Antwort (Basismodell):")
print(before_text)

In [ ]:
# 5. Datensatz laden und tokenisieren
from datasets import load_dataset

# Deutscher Instruktions-Datensatz: 3.721 Frage-Antwort-Paare aus OpenAssistant
data = load_dataset("mayflowergmbh/oasst_de", split="train")
print(f"Datensatz: {len(data)} Einträge")

# Tokenisierung mit Chat-Template und token_type_ids.
# Gemma 3 benötigt token_type_ids (Multimodal-Architektur: Text vs. Bild).
# Für reines Text-Training: token_type_ids = 0 (alles Text-Token).
def tokenize(example):
    user_msg = example["instruction"]
    if example.get("input"):
        user_msg += "\n" + example["input"]
    messages = [
        {"role": "user", "content": user_msg},
        {"role": "assistant", "content": example["output"]}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    tokens = tokenizer(text, truncation=True, max_length=512)
    tokens["token_type_ids"] = [0] * len(tokens["input_ids"])
    return tokens

data = data.map(tokenize, remove_columns=data.column_names)
print(f"Tokenisiert: {len(data)} Einträge")

In [ ]:
# 6. Training (QLoRA Fine-Tuning)
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Data-Collator: Padding + Labels für das Training
def gemma3_collator(features):
    batch = tokenizer.pad(features, padding=True, return_tensors="pt")
    batch["labels"] = batch["input_ids"].clone()
    if "token_type_ids" not in batch:
        batch["token_type_ids"] = torch.zeros_like(batch["input_ids"])
    return batch

trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    data_collator=gemma3_collator,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=30,  # Für Demo-Zwecke. Für echtes Training deutlich erhöhen.
        learning_rate=2e-4,
        bf16=True,
        logging_steps=10,
        output_dir="outputs",
    ),
    peft_config=lora_config,
)

trainer.train()

## Ergebnis des Trainings

**Woran erkennt man, dass das Training funktioniert hat?**

Der wichtigste Indikator ist der **Training-Loss**: Er misst, wie gut das Modell die Trainingsdaten vorhersagen kann. Ein sinkender Loss bedeutet, dass das Modell die Muster im Datensatz lernt.

- **Typischer Verlauf:** Der Loss startet bei ~2.5–3.0 und sinkt über die 30 Steps auf ~1.6–1.8.
- **Was das bedeutet:** Das Modell hat gelernt, deutsche Instruktions-Antworten im Stil des Datensatzes (`oasst_de`) besser vorherzusagen.

**Warum sehen VORHER und NACHHER trotzdem ähnlich aus?**

Gemma 3 4B ist bereits ein leistungsfähiges Modell, das Deutsch gut beherrscht. Mit nur 30 Training-Steps (eine Demo-Einstellung) sind die Änderungen subtil — etwa leichte Unterschiede in Wortwahl, Satzstruktur oder Antwortaufbau. Für deutlich sichtbare Unterschiede bräuchte man:
- **Mehr Training-Steps** (300–1.000+)
- **Einen spezialisierten Datensatz** (z.B. medizinische Fachtexte, juristischer Stil)

Das Notebook demonstriert den **technischen Workflow** von QLoRA — derselbe Prozess skaliert auf produktionsreife Ergebnisse, wenn man Datensatz und Trainingszeit anpasst.

In [ ]:
# 7. NACHHER-Test — Antwort nach dem Fine-Tuning
after_text = generate(test_prompt)

print("=" * 60)
print("VERGLEICH: VORHER vs. NACHHER")
print("=" * 60)
print(f"\nPrompt: {test_prompt}")
print(f"\n--- VORHER (Basismodell) ---\n{before_text}")
print(f"\n--- NACHHER (Fine-Tuned) ---\n{after_text}")

In [ ]:
# 8. (Optional) Adapter speichern
# trainer.save_model("my_slm_adapter")